<a href="https://colab.research.google.com/github/johnsonomar01/avatarify-python/blob/omar/Avatarify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avatarify Colab Server

This Colab notebook is for running Avatarify rendering server. It allows you to run Avatarify on your computer **without GPU** in this way:

1. When this notebook is executed, it starts listening for incoming requests from your computer;
1. You start the client on your computer and it connects to the notebook and starts sending requests;
1. This notebooks receives the requests from your computer, renders avatar images and sends them back;

To this end, all the heavy work is offloaded from your computer to this notebook so you don't need to have a beafy hardware on your PC anymore.


## Start the server
Run the cells below (Shift+Enter) sequentially and pay attention to the hints and instructions included in this notebook.

At the end you will get a command for running the client on your computer.

## Start the client

Make sure you have installed the latest version of Avatarify on your computer. Refer to the [README](https://github.com/alievk/avatarify#install) for the instructions.

When it's ready execute this notebook and get the command for running the client on your computer.


### Technical details

The client on your computer connects to the server via `ngrok` TCP tunnel or a reverse `ssh` tunnel.

`ngrok`, while easy to use, can induce a considerable network lag ranging from dozens of milliseconds to a second. This can lead to a poor experience.

A more stable connection could be established using a reverse `ssh` tunnel to a host with a public IP, like an AWS `t3.micro` (free) instance. This notebook provides a script for creating a tunnel, but launching an instance in a cloud is on your own (find the manual below).

# Install

### Avatarify
Follow the steps below to clone Avatarify and install the dependencies.

In [1]:
!cd /content
!rm -rf *

In [2]:
!git clone https://github.com/alievk/avatarify.git

Cloning into 'avatarify'...
remote: Enumerating objects: 1477, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1477 (delta 20), reused 23 (delta 6), pack-reused 1433
Receiving objects: 100% (1477/1477), 5.66 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (953/953), done.


In [3]:
cd avatarify

/content/avatarify


In [4]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy pyyaml==5.3.1

Cloning into 'fomm'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (211/211), 58.16 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 9.6 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp38-cp38-linux_x86_64.whl size=44634 sha256=18bc65b0327a0af6c26aa8e4ace3180a0c83ef2e58b469f618b4b47d4f26d020
  Stored in directory: /root/.cache/pip/wheels/13/90/db/290ab3a34f2ef0b5a0f89235dc2d40fea83e77de84ed2dc05c
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [5]:
!scripts/download_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228M  100  228M    0     0  10.9M      0  0:00:20  0:00:20 --:--:-- 12.7M
Expected checksum: 8a45a24037871c045fbb8a6a8aa95ebc
Found checksum:    8a45a24037871c045fbb8a6a8aa95ebc  vox-adv-cpk.pth.tar


### ngrok
Follow the steps below to setup ngrok. You will also need to sign up on the ngrok site and get your authtoken (free).


In [6]:
# Download ngrok
!scripts/get_ngrok.sh

ngrok is not found, installing...
Done!


# Run
Start here if the runtime was restarted after installation.

In [7]:
cd /content/avatarify

/content/avatarify


In [8]:
#!git pull origin

In [9]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [10]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Start the worker


In [11]:
# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

This command should print lines if the worker is successfully started

In [12]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root         809 15.4  2.4 2677144 322256 ?      S    06:30   0:01 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         836  0.0  1.2 2685340 169548 ?      Sl   06:30   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         837  0.0  1.2 2677144 167532 ?      S    06:30   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         840  0.0  1.2 2685340 170108 ?      Sl   06:30   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-po

# Open ngrok tunnel

#### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [13]:
# Paste your authtoken here in quotes
authtoken = "2JBB3WnoJcHUswmKunk5dnBrdgh_6etqpeiXXyZYHan98VZdx"

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [14]:
# Set your region here in quotes
region = "eu"

In [15]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [16]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [17]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

tcp://6.tcp.eu.ngrok.io:14464 -> 5557 [input]
tcp://7.tcp.eu.ngrok.io:16957 -> 5558 [output]
Tunnel opened


### [Optional] AWS proxy
Alternatively you can create a ssh reverse tunnel to an AWS `t3.micro` instance (it's free). It has lower latency than ngrok.

1. In your AWS console go to Services -> EC2 -> Instances -> Launch Instance;
1. Choose `Ubuntu Server 18.04 LTS` AMI;
1. Choose `t3.micro` instance type and press Review and launch;
1. Confirm your key pair and press Launch instances;
1. Go to the security group of this instance and edit inbound rules. Add TCP ports 5557 and 5558 and set Source to Anywhere. Press Save rules;
1. ssh into the instance (you can find the command in the Instances if you click on the Connect button) and add this line in the end of `/etc/ssh/sshd_config`:
```
GatewayPorts yes
```
then restart `sshd`
```
sudo service sshd restart
```
1. Copy your `key_pair.pem` by dragging and dropping it into avatarify folder in this notebook;
1. Use the command below to open the tunnel;
1. Start client with a command (substitute `run_mac.sh` with `run_windows.bat` or `run.sh`)
```
./run_mac.sh --is-client --in-addr tcp://instace.compute.amazonaws.com:5557 --out-addr tcp://instance.compute.amazonaws.com:5558
```

In [18]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Start the client
When you run the cell below it will print a command. Run this command on your computer:

1. Open a terminal (in Windows open `Anaconda Prompt`);
2. Change working directory to the `avatarify` directory:</br>
* Windows (change `C:\path\to\avatarify` to your path)</br>
`cd C:\path\to\avatarify`</br></br>
* Mac/Linux (change `/path/to/avatarify` to your path)</br>
`cd /path/to/avatarify`
3. Copy-paste to the terminal the command below and run;
4. It can take some time to connect (usually up to 10 seconds). If the preview window doesn't appear in a minute or two, look for the errors above in this notebook and report in the [issues](https://github.com/alievk/avatarify/issues) or [Slack](https://join.slack.com/t/avatarify/shared_invite/zt-dyoqy8tc-~4U2ObQ6WoxuwSaWKKVOgg).

In [19]:
print('Copy-paste to the terminal the command below and run (press Enter)\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

Copy-paste to the terminal the command below and run (press Enter)

Mac:
./run_mac.sh --is-client --in-addr tcp://6.tcp.eu.ngrok.io:14464 --out-addr tcp://7.tcp.eu.ngrok.io:16957

Windows:
run_windows.bat --is-client --in-addr tcp://6.tcp.eu.ngrok.io:14464 --out-addr tcp://7.tcp.eu.ngrok.io:16957

Linux:
./run.sh --is-client --in-addr tcp://6.tcp.eu.ngrok.io:14464 --out-addr tcp://7.tcp.eu.ngrok.io:16957


# Logs

If something doesn't work as expected, please run the cells below and include the logs in your report.

In [ ]:
#@title
!cat ./var/log/cam_fomm.log | head -100

In [ ]:
#@title
!cat ./var/log/recv_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/predictor_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/send_worker.log | tail -100